In [1]:
import os
import open3d as o3d
from pathlib import Path 
import glob 
from scipy.sparse import csr_matrix 
from scipy.sparse.csgraph import connected_components
import numpy as np
import copy

INFO - 2025-04-17 13:14:53,714 - utils - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2025-04-17 13:14:53,715 - utils - NumExpr defaulting to 8 threads.


In [2]:
# doesnot remove non-connected smaller components from teeth_gums_mesh

# def extract_teeth_and_gums(mesh, area_threshold=None, percentile_threshold=95):
#     """
#     Extract teeth and gums from a dental mesh by removing large triangles.
    
#     Parameters:
#     -----------
#     mesh : o3d.geometry.TriangleMesh
#         The original mesh containing teeth, gums, and base
#     area_threshold : float, optional
#         Maximum triangle area to keep (triangles larger than this will be removed)
#         If None, percentile_threshold will be used instead
#     percentile_threshold : float, optional
#         Percentile value used to automatically determine area_threshold
#         Only used if area_threshold is None
        
#     Returns:
#     --------
#     teeth_gums_mesh : o3d.geometry.TriangleMesh
#         Mesh containing only teeth and gums (small triangles)
#     base_mesh : o3d.geometry.TriangleMesh
#         Mesh containing only the base structure (large triangles)
#     """
#     # Get triangles and vertices
#     vertices = np.asarray(mesh.vertices)
#     triangles = np.asarray(mesh.triangles)
    
#     # Calculate area of each triangle
#     triangle_areas = []
#     for triangle in triangles:
#         # Get vertices of the triangle
#         v0 = vertices[triangle[0]]
#         v1 = vertices[triangle[1]]
#         v2 = vertices[triangle[2]]
        
#         # Calculate two edges of the triangle
#         edge1 = v1 - v0
#         edge2 = v2 - v0
        
#         # Calculate area using cross product
#         cross_product = np.cross(edge1, edge2)
#         area = 0.5 * np.linalg.norm(cross_product)
#         triangle_areas.append(area)
    
#     triangle_areas = np.array(triangle_areas)
    
#     # Determine threshold if not provided
#     if area_threshold is None:
#         area_threshold = np.percentile(triangle_areas, percentile_threshold)
#         print(f"Automatically determined area threshold: {area_threshold}")
    
#     # Create mask for small triangles (teeth and gums)
#     small_triangle_mask = triangle_areas < area_threshold
    
#     # Get triangles for teeth and gums
#     teeth_gums_triangles = triangles[small_triangle_mask]
    
#     # Get triangles for base
#     base_triangles = triangles[~small_triangle_mask]
    
#     # Create new mesh for teeth and gums
#     teeth_gums_mesh = o3d.geometry.TriangleMesh()
#     teeth_gums_mesh.vertices = o3d.utility.Vector3dVector(vertices)
#     teeth_gums_mesh.triangles = o3d.utility.Vector3iVector(teeth_gums_triangles)
    
#     # Create new mesh for base
#     base_mesh = o3d.geometry.TriangleMesh()
#     base_mesh.vertices = o3d.utility.Vector3dVector(vertices)
#     base_mesh.triangles = o3d.utility.Vector3iVector(base_triangles)
    
#     # Remove unreferenced vertices
#     teeth_gums_mesh.remove_unreferenced_vertices()
#     base_mesh.remove_unreferenced_vertices()
    
#     # Compute normals
#     teeth_gums_mesh.compute_vertex_normals()
#     base_mesh.compute_vertex_normals()
    
#     return teeth_gums_mesh, base_mesh

In [ ]:
# remove non-connected smaller components from teeth_gums_mesh
def extract_teeth_and_gums(mesh, area_threshold=None, percentile_threshold=95):
    """
    Extract teeth and gums from a dental mesh by removing large triangles.
    Only keeps the largest connected component of the teeth and gums.
    
    Parameters:
    -----------
    mesh : o3d.geometry.TriangleMesh
        The original mesh containing teeth, gums, and base
    area_threshold : float, optional
        Maximum triangle area to keep (triangles larger than this will be removed)
        If None, percentile_threshold will be used instead
    percentile_threshold : float, optional
        Percentile value used to automatically determine area_threshold
        Only used if area_threshold is None
        
    Returns:
    --------
    teeth_gums_mesh : o3d.geometry.TriangleMesh
        Mesh containing only teeth and gums (small triangles), only largest connected component
    base_mesh : o3d.geometry.TriangleMesh
        Mesh containing only the base structure (large triangles)
    """
    # Get triangles and vertices
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    
    # Calculate area of each triangle
    triangle_areas = []
    for triangle in triangles:
        # Get vertices of the triangle
        v0 = vertices[triangle[0]]
        v1 = vertices[triangle[1]]
        v2 = vertices[triangle[2]]
        
        # Calculate two edges of the triangle
        edge1 = v1 - v0
        edge2 = v2 - v0
        
        # Calculate area using cross product
        cross_product = np.cross(edge1, edge2)
        area = 0.5 * np.linalg.norm(cross_product)
        triangle_areas.append(area)
    
    triangle_areas = np.array(triangle_areas)
    
    # Determine threshold if not provided
    if area_threshold is None:
        area_threshold = np.percentile(triangle_areas, percentile_threshold)
        print(f"Automatically determined area threshold: {area_threshold}")
    
    # Create mask for small triangles (teeth and gums)
    small_triangle_mask = triangle_areas < area_threshold
    
    # Get triangles for teeth and gums
    teeth_gums_triangles = triangles[small_triangle_mask]
    
    # Get triangles for base
    base_triangles = triangles[~small_triangle_mask]
    
    # Create new mesh for teeth and gums
    teeth_gums_mesh = o3d.geometry.TriangleMesh()
    teeth_gums_mesh.vertices = o3d.utility.Vector3dVector(vertices)
    teeth_gums_mesh.triangles = o3d.utility.Vector3iVector(teeth_gums_triangles)
    
    # Create new mesh for base
    base_mesh = o3d.geometry.TriangleMesh()
    base_mesh.vertices = o3d.utility.Vector3dVector(vertices)
    base_mesh.triangles = o3d.utility.Vector3iVector(base_triangles)
    
    # Remove unreferenced vertices
    teeth_gums_mesh.remove_unreferenced_vertices()
    base_mesh.remove_unreferenced_vertices()
    
    # Extract only the largest connected component from teeth_gums_mesh
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        triangle_clusters, cluster_n_triangles, cluster_area = teeth_gums_mesh.cluster_connected_triangles()
    
    triangle_clusters = np.asarray(triangle_clusters)
    cluster_n_triangles = np.asarray(cluster_n_triangles)
    cluster_area = np.asarray(cluster_area)
    
    if len(cluster_n_triangles) > 0:  # If there are clusters
        # Find the largest cluster
        largest_cluster_idx = np.argmax(cluster_n_triangles)
        
        # Create a mask for triangles in the largest cluster
        largest_cluster_triangles = np.asarray(teeth_gums_mesh.triangles)
        largest_cluster_mask = triangle_clusters == largest_cluster_idx
        
        # Create a new mesh with only the largest cluster
        largest_cluster_mesh = o3d.geometry.TriangleMesh()
        largest_cluster_mesh.vertices = teeth_gums_mesh.vertices
        largest_cluster_mesh.triangles = o3d.utility.Vector3iVector(
            largest_cluster_triangles[largest_cluster_mask])
        
        # Remove unreferenced vertices
        largest_cluster_mesh.remove_unreferenced_vertices()
        
        # Replace the teeth_gums_mesh with the largest cluster mesh
        teeth_gums_mesh = largest_cluster_mesh
    
    # Compute normals
    teeth_gums_mesh.compute_vertex_normals()
    base_mesh.compute_vertex_normals()
    
    return teeth_gums_mesh, base_mesh

In [ ]:
file = "test_big.ply"
mesh = o3d.io.read_triangle_mesh(file)

teeth_gums_mesh, base_mesh = extract_teeth_and_gums(mesh, area_threshold=None, percentile_threshold=99)
o3d.io.write_triangle_mesh("separated_teeth_gums_base/teeth_gums.ply", teeth_gums_mesh)
o3d.io.write_triangle_mesh("separated_teeth_gums_base/base.ply", base_mesh)

Automatically determined area threshold: 0.12873196912338164


True